In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pandas as pd
import mediapipe as mp

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [6]:
dataset, dataname = [],[]
root = 'dataset'
ctr = 0
classes = []
alexnet = False
size = 224
for alphabet in os.listdir(root):
    limit =0 
    print(alphabet,end=' ')
    classes += [alphabet]
    for img in os.listdir(f'{root}/{alphabet}'):
        img = cv2.imread(f'{root}/{alphabet}/{img}')
        img = cv2.resize(img,(size,size))
        dataset += [img]
        dataname += [alphabet]
        limit+=1
        if(limit > 500):
            break
        
    ctr+=1


dataset = np.array(dataset)
dataname = np.array(dataname)

A B C D E F G H I J K L M N O P Q R S space T U V W X Y Z 

In [8]:
# augment
images = zip(dataset,dataname)

augmented_img = []
augmented_class = []
for (img, className) in images:
    flipped = cv2.flip(img,1)
    # plt.imshow(img)
    # plt.show()
    aug = flipped
    aug = np.array(aug)
    # plt.imshow(aug)
    # plt.show()
    augmented_img.append(aug)   
    augmented_class.append(className)
    # break
# a,_ = images
# print(len(a))
print("before",len(dataset))
dataset = np.concatenate((dataset, augmented_img))
dataname = np.concatenate((dataname, augmented_class))
print(len(dataset))




before 13527
27054


In [ ]:
hands_arr = []
images = zip(dataset,dataname)
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.4) as hands:
    # print(images)
    for idx, (img,className) in enumerate(images):
        results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        # print(results)
        if not results.multi_hand_landmarks:
          continue
        hand_landmarks = results.multi_hand_landmarks
        print(hand_landmarks)

        if len(hand_landmarks) > 0:
            hand_landmarks = hand_landmarks[0]
        else:
            hand_landmarks = None
        
        # print(hand_landmarks)
        hand_data = {"Class": className}
        image_height, image_width, _ = img.shape

        x_max = 0
        y_max = 0
        x_min = image_width
        y_min = image_height

        for idx1, landmark in enumerate(hand_landmarks.landmark):
            relative_x = landmark.x - hand_landmarks.landmark[0].x
            relative_y = landmark.y - hand_landmarks.landmark[0].y

            hand_data[f"Point{idx1 + 1}_X"] = relative_x 
            hand_data[f"Point{idx1 + 1}_Y"] = relative_y
        hands_arr.append(hand_data)
        print(idx)





        



In [13]:
df = pd.DataFrame(hands_arr)
display(df.head(5))

print(df['Point1_X'].value_counts())
print(df['Point1_Y'].value_counts())

df.drop(['Point1_X','Point1_Y'],axis=1,inplace=True)
df.to_csv('./output/dataset.csv', index=False)

,Class,Point1_X,Point1_Y,Point2_X,Point2_Y,Point3_X,Point3_Y,Point4_X,Point4_Y,Point5_X,...,Point17_X,Point17_Y,Point18_X,Point18_Y,Point19_X,Point19_Y,Point20_X,Point20_Y,Point21_X,Point21_Y
0,A,0.0,0.0,0.113495,-0.082870,0.181338,-0.214224,0.191743,-0.324273,0.176989,...,0.009401,-0.114414,-0.131490,-0.251106,-0.068488,-0.264602,-0.054249,-0.181575,-0.055466,-0.128653
1,A,0.0,0.0,0.117747,-0.071520,0.192580,-0.206726,0.203706,-0.324133,0.199602,...,0.026212,-0.104201,-0.124266,-0.265737,-0.052779,-0.257111,-0.040167,-0.167494,-0.040408,-0.110646
2,A,0.0,0.0,0.074529,-0.056717,0.124976,-0.161563,0.136414,-0.249405,0.132418,...,-0.032550,-0.106327,-0.095463,-0.181809,-0.096761,-0.221975,-0.089207,-0.164900,-0.080041,-0.118804
3,A,0.0,0.0,0.116533,-0.095569,0.184809,-0.251257,0.181613,-0.380073,0.169156,...,-0.001719,-0.126659,-0.181320,-0.265591,-0.107658,-0.301670,-0.078174,-0.204446,-0.072207,-0.136792
4,A,0.0,0.0,0.115344,-0.095472,0.185029,-0.257217,0.183975,-0.386701,0.171799,...,-0.000573,-0.125995,-0.177708,-0.268218,-0.108086,-0.303106,-0.079395,-0.207734,-0.070093,-0.138574


Point1_X
0.0    25437
Name: count, dtype: int64
Point1_Y
0.0    25437
Name: count, dtype: int64


In [14]:
df['Class'].value_counts()

Class
K        1002
S        1002
Y        1000
Z         996
O         995
F         994
P         992
D         990
R         988
C         980
I         972
H         970
Q         967
M         964
G         952
T         942
J         939
V         928
E         927
U         921
A         918
X         915
L         912
B         900
W         869
space     855
N         647
Name: count, dtype: int64

In [15]:
df.duplicated().sum()

0